### Student Information
Name:楊哲銘

Student ID: 113753219

GitHub ID: jhemingy

Kaggle name: Jhe Ming Yang

Kaggle private scoreboard snapshot: 

---

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home exercises** in the [DM2024-Lab2-master Repo](https://github.com/didiersalazar/DM2024-Lab2-Master). You may need to copy some cells from the Lab notebook to this notebook. 


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/competitions/dm-2024-isa-5810-lab-2-homework) regarding Emotion Recognition on Twitter by this link: https://www.kaggle.com/competitions/dm-2024-isa-5810-lab-2-homework. The scoring will be given according to your place in the Private Leaderboard ranking: 
    - **Bottom 40%**: Get 20% of the 30% available for this section.

    - **Top 41% - 100%**: Get (0.6N + 1 - x) / (0.6N) * 10 + 20 points, where N is the total number of participants, and x is your rank. (ie. If there are 100 participants and you rank 3rd your score will be (0.6 * 100 + 1 - 3) / (0.6 * 100) * 10 + 20 = 29.67% out of 30%.)   
    Submit your last submission **BEFORE the deadline (Nov. 26th, 11:59 pm, Tuesday)**. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the **img** folder of this repository and rerun the cell **Student Information**.
    

3. Third: __This part is worth 30% of your grade.__ A report of your work developing the model for the competition (You can use code and comment on it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained. 


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook**.


Upload your files to your repository then submit the link to it on the corresponding e-learn assignment.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Nov. 26th, 11:59 pm, Tuesday)__. 

In [ ]:
### Begin Assignment Here

###Part 1 prepare data###
#First, I merged all the data into a single DataFrame called emotionmerged_df, which was then split into test and train DataFrames.

# Read data
data = []
with open('/kaggle/input/dm-2024-isa-5810-lab-2-homework/tweets_DM.json', 'r') as f:
    for line in f:
        data.append(json.loads(line))
 f.close()

emotion = pd.read_csv('/kaggle/input/dm-2024-isa-5810-lab-2-homework/emotion.csv')

data_identification = pd.read_csv('/kaggle/input/dm-2024-isa-5810-lab-2-homework/data_identification.csv')

tweet_df = pd.DataFrame([
    {
        'tweet_id': item['_source']['tweet']['tweet_id'],
        'text': item['_source']['tweet']['text'],
        'hashtags': item['_source']['tweet']['hashtags'],
        'crawldate': item['_crawldate']
    }
    for item in data
])

emotionmerged_df = pd.merge(tweet_df, emotion, on='tweet_id', how='inner')
emotionmerged_df=pd.merge(data_identification,emotionmerged_df,  on='tweet_id', how='left')

emotionmerged_df_test=emotionmerged_df[emotionmerged_df['identification']=='test']
emotionmerged_df_train=emotionmerged_df[emotionmerged_df['identification']=='train']



###Part 2: Use llm model to get embeddings###
#The distilgpt2 model was used to generate embeddings for the training set

#load llm model
from transformers import AutoTokenizer, AutoModel
import torch
model_name = "distilgpt2" 
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).cuda()
from torch.nn import DataParallel
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs!")
    model = DataParallel(model)

model =model.eval()

#get embeddings and save as partquet file
from tqdm import tqdm
def get_batch_embeddings(texts, batch_size=128):
    total = len(texts)
    percent_step = total // 100 if total >= 100 else 1  # 每 1% 的步长
    embeddings = []
    tokenizer.pad_token = tokenizer.eos_token
    with tqdm(total=100, desc="Generating Embeddings (1% steps)") as pbar:
        for i in range(0, total, batch_size):
            batch_texts = texts[i:i + batch_size]
            inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=2048).to("cuda")
            with torch.no_grad():
                outputs = model(**inputs, output_hidden_states=True)
            batch_embeddings = outputs.hidden_states[-1].mean(dim=1)
            embeddings.append(batch_embeddings.cpu().numpy())
            if (i + batch_size) // percent_step > i // percent_step:
                pbar.update((i + batch_size) // percent_step - i // percent_step)

    return np.vstack(embeddings)

train_texts = emotionmerged_df_train['text'].tolist()  # Extract texts
train_embeddings = get_batch_embeddings(train_texts)

# Combine embeddings with tweet IDs
train_tweet_ids = emotionmerged_df_train['tweet_id'].tolist()
embeddings_df = pd.DataFrame(train_embeddings)
embeddings_df.insert(0, 'tweet_id', train_tweet_ids)  # Add tweet_id as the first column

# Save the dataframe as a Parquet file
parquet_file_path = 'train_embeddings.parquet'
embeddings_df.to_parquet(parquet_file_path, engine='pyarrow', index=False)



###Part 3: Use faiss to search the top k simular embedding vectors ###
#After have all the embeddings, I upload the file as a kaggle dataset, then i store them to the faiss
import faiss
embedding_df = pd.read_parquet('/kaggle/input/embedding/embedding.parquet')
embedding_df.head()
embedding_df = pd.merge(embedding_df, emotion, on='tweet_id', how='inner')

embeddings = np.array(embedding_df['embedding'].tolist())
tweet_ids = embedding_df['tweet_id'].tolist()  
emotions = embedding_df['emotion'].tolist() 
print(f"Embedding shape: {embeddings.shape}")

embedding_dim = embeddings.shape[1] 
index = faiss.IndexFlatL2(embedding_dim)  
faiss.normalize_L2(embeddings)  
index.add(embeddings)  


#Then for all test data's text, I translate it to embedding vectors
#Use faiss to search the top 7 simular vectors and their emotions

def search_faiss(query_vectors, top_k=7):

    faiss.normalize_L2(query_vectors)
    distances, indices = index.search(query_vectors, top_k)
    results = []
    for query_idx, neighbors in enumerate(indices):
        result = {
            "query_idx": query_idx,
            "neighbors": [
                {
                    "index": neighbor_idx,
                    "tweet_id": tweet_ids[neighbor_idx],  # 返回对应的 tweet_id
                    "emotions": emotions[neighbor_idx],  # 返回情绪
                    "distance": distances[query_idx][neighbor_rank],
                }
                for neighbor_rank, neighbor_idx in enumerate(neighbors)
            ],
        }
        results.append(result)
    return results

def results_to_dataframe(results):
    data = []
    for result in results:
        query_idx = result['query_idx']
        for neighbor in result['neighbors']:
            data.append({
                "query_idx": query_idx,
                "tweet_id": neighbor["tweet_id"],
                "emotion": neighbor["emotions"],
                "distance": neighbor["distance"]
            })
    return pd.DataFrame(data)

results = search_faiss(query_embeddings, top_k=7)
results_df = results_to_dataframe(results)

###Part 4: Vote the simular embedding's emotions to get the results ###


def perform_voting(results_df):
    voting_results = results_df.groupby('query_idx')['emotion'].apply(
        lambda x: x.value_counts().idxmax()  
    ).reset_index()
    voting_results.columns = ['query_idx', 'voted_emotion']  
    return voting_results

query_embeddings = get_batch_embeddings(emotionmerged_df_test['text'].to_list())  
results = search_faiss(query_embeddings, top_k=7)

results_df = results_to_dataframe(results)
voting_df = perform_voting(results_df)

y_test_pred=voting_df['voted_emotion'].values
submission = pd.DataFrame({
    'id': emotionmerged_df_test['tweet_id'],
    'emotion': y_pred_labels
})


# I was too late to finish the Kaggle competition.
# I used a random sample of 1,000 records from the training set to generate the results.

# Accuracy: 0.5220
# Precision: 0.5139
# Recall: 0.5220
# F1 Score: 0.4985

# Classification Report:
#               precision    recall  f1-score   support

#        anger       0.60      0.27      0.37        22
# anticipation       0.53      0.46      0.49       160
#      disgust       0.35      0.26      0.30        92
#         fear       0.40      0.21      0.28        47
#          joy       0.55      0.81      0.66       364
#      sadness       0.46      0.40      0.43       139
#     surprise       0.50      0.17      0.26        23
#        trust       0.58      0.35      0.43       153

#     accuracy                           0.52      1000
#    macro avg       0.50      0.37      0.40      1000
# weighted avg       0.51      0.52      0.50      1000
